INTRODUCTION: 

The purpose of this notebook, along with 02_run_example.ipynb, is to provide
a tutorial of how you may want to use the pop_exp pacakge functions.

This notebook cleans some raw data files for use in the pop_exp functions, 
and then 02_run_example.ipynb demonstrates how to use the pop_exp functions.

Prerequisites: This tutorial for pop_exp assumes that you have a version of 
Python installed on your computer compatible with the requirements of pop_exp, 
you have an IDE, and you’re able to open and run a Jupyter notebook as well as 
Python scripts. It also assumes that you are able to create and manage a virtual 
environment into which you have installed the Python package pop_exp. 

WHAT WE ARE DOING HERE: 

This tutorial will teach you how to use pop exp to find the number of people 
affected by any wildfire disaster by ZCTA across the years 2016-2018. It will 
disucss in more detail how pop_exp allows you to define exposure in different 
ways shortly.  

pop_exp is for population exposure. 

DATA USED IN THIS TUTORIAL:

The functions in pop_exp allow the user to find the number of people who reside 
near environmental hazards like wildfires, either inside the hazard boundaries 
or within a buffer of the environmental hazard, using a dataset of hazards and 
a gridded population dataset. If the user wishes to find the number of people 
affected within additional spatial units such as ZCTAs or census tracts, a 
dataset describing those geographies is also required. 

In this example, we'll use a publicly available dataset of US wildfire disaster 
boundaries for the years 2016-2018 as our hazard data. A description of this 
data and a link for download is available here: 

https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/R73R85

This tutorial will demonstrate how to find the number of people residing within
a buffer of these wildfires by ZCTA, so we'll also use a shapefile of 2020 
ZCTAs, described in detail and available for download here:

https://www.census.gov/programs-surveys/geography/guidance/geo-areas/zctas.html

For the required gridded population data, which is used by the function to 
determine how many people live where, we'll use the version of the Global Human
Settlement Layer which describes the residential population of the globe at 
100 m resolution. It's downloadable here: 

https://human-settlement.emergency.copernicus.eu/download.php?ds=pop

GOALS OF FUNCTION USE: 

The Python package pop_exp can do five distinct computations. 

1. Find the total number of people who reside within a buffer distance (which 
can be 0) of *any* environmental hazard in a set of hazards.
2. Find the total number of people who reside within a buffer distance (which 
can be 0) of *EACH* environmental hazard in a set of hazards.
3. Find the total number of people who reside within a buffer distance (which 
can be 0) of *any* environmental hazard in a set of hazards, by additional spatial 
unit (ex. the total number of people who resided within 10km of any wildfire 
disaster in 2018 by ZCTA).
4. Find the total number of people who reside within a buffer distance (which 
can be 0) of *EACH* environmental hazard in a set of hazards, by additional 
spatial unit. 
5. Find the number of people residing within each spatial unit according to a
gridded population dataset. 

The fifth function is meant to provide denominators for computations conducted 
with the first 4. For example, you may want to find the total number of people 
who resided within 10km of any wildfire disaster in 2018 by ZCTA, and then 
calculate the proportion of the ZCTA population that was exposed. To do this, 
you could use a function in pop_exp to find the ZCTA population according to 
the gridded population raster you used to determine exposure. 

This tutorial will demonstrate all of these options. 


To demo all the functions available in pop_exp, we will do five separate things,
which align with the five options available in the package. 

1. Find the total number of people residing within 10km of *any* US wildfire 
disaster in 2016, 2017, and 2018. 
2. Find the total number of people residing within 10 km of *EACH* US wildfire
disaster in 2016, 2017, and 2018.
3. Find the total number of people residing within 10km of *any* US wildfire 
disaster in 2016, 2017, and 2018 by 2020 ZCTA. 
4. Find the total number of people residing within 10 km of *EACH* US wildfire
disaster in 2016, 2017, and 2018 by 2020 ZCTA.
5. Find the population of all 2020 ZCTAs. 

To do the first two computations, we need to call the pop_exp function 
find_number_of_people_affected. The function parameter by_unique_hazard allows
us to specify whether we want to find the total number of people residing near 
*any* hazard, or *each* hazard. When by_unique_hazard = False, the function 
computes the number of people exposed to any hazard. When it's True, it computes
a total for each unique hazard. 

To do the second computations, we'll call the function 
find_number_of_people_affected_by_geo. As with find_number_of_people_affected, 
when by_unique_hazard = False, the function computes the number of people 
exposed to any hazard by ZCTA, and when it's True, it computes
a total number of people affected for each unique hazard by ZCTA. 

DATA CLEANING:

To prepare to do the above computations, we need to prepare the data. 
This is where we start coding! First we import some libraries. 

In [ ]:
import pathlib
import sys
import matplotlib.pyplot as plt
import geopandas as gpd

We'll start by preparing the wildfire data. We'll set directories, and then 
read in the raw wildfire data as downloaded from Harvard dataverse. Note that
the raw data contains wildfire disasters for years 2000-2019, which is a lot, 
and we're going to filter down to only 2016-2018 for this tutorial. 

In [ ]:
base_path = pathlib.Path.cwd().parent
data_dir = base_path / "demo_data"

fires = gpd.read_file(data_dir / "01_raw_data"/ "wildfires_conus.geojson")

We'll plot the data to make sure it read in correctly.

In [ ]:
fires.plot()

For both find_num_people_affected, and find_number_of_people_affected_by_geo, 
when we run the functions, we need to pass a path to a hazard dataframe with 
3 columns:  ID_climate_hazard, buffer_dist, and geometry. So we need to decide 
on a buffer distance and create that column, and rename the other columns to 
the correct names. 

For this tutorial, I've decided I want to consider people exposed to a wildfire
if they live within 10 km of the boundaries of the wildfire disasters that are
specified in my dataset. You could do something different if you think the 
relevant distance from your hazard is different. You can also assign a buffer
of 0 to your hazards, or different buffers to each hazard in your dataset. They
don't all have to be the same.  

The buffer distance is in meters, so we'll specify a 10,000 m buffer distance. 

In [ ]:
fires["buffer_dist"] = 10000 # buffer distance in in meters 
fires.head # Checking what columns I have in the data 

We've created a buffer distance column. We need to select and rename the
remaining columns we need for pop_exp, but we also need to select the years 
we're interested in. 

I'm interested in years 2016-2018, and I've decided that I want to determine 
exposure by year. I want to compute the total number of people affected by any 
fire in 2016, 2017, and 2018, as well as the three other exposure definitions we
wrote out above. 

There is no option in pop_exp to indicate which hazards are
for which year, or time period. If I want to know the total number of people 
affected by hazards in 2016 but not 2017, I need to feed pop_exp the exposure
data for 2016 along with a gridded population dataset that represents the 
population in 2016. If I wanted monthly exposure for 2016, I'd need to split
my exposure data up by month and call the function separately on each month. 

So before selecting just the ID, hazard, and buffer distance columns, I'm 
going to select and split up the years I'm interested in. 

In [ ]:
# Select fires in 2016, 2017, 2018
fires = fires[fires["wildfire_year"].isin([2016, 2017, 2018])]
# Split this into a list of dataframes by year
fires_by_year = [fires[fires["wildfire_year"] == year] for year in [2016, 2017, 2018]]

Now that I have my exposure datasets, I'll select and rename the columns I need
for pop_exp: ID_climate_hazard, buffer_dist, and geometry. 

In [ ]:
# First, select cols
fires_by_year = [fire[["wildfire_id", "buffer_dist", "geometry"]] for fire in fires_by_year]
# Then rename the wildfire ID col
fires_by_year = [fire.rename(columns={"wildfire_id": "ID_climate_hazard"}) for fire in fires_by_year]

Finally, I'm going to write these out into an interim data folder to call 
using the pop_exp function, since the pop_exp function requires you to pass 
a path name. I'm using geojson files because that's what the function requires. 
Parquet is also an option. 

In [ ]:
for i, fire in enumerate(fires_by_year):
    fire.to_file(data_dir / "02_interim_data" / f"wildfires_{2016 + i}.geojson", driver="GeoJSON")

We've dealt with the wildfire disaster exposure data which is going to be our
"hazard" data for pop exp. Now we also need to get the ZCTA data into the right
format. 

I'm using 2020 ZCTA data, since my years are closer to the 2020 census than the
2010 census. I'm going to read in the data, and then select and rename the 
columns to be what the functions find_number_of_people_affected and 
find_number_of_people_affected_by_geo require. 

I need to rename the ZCTA ID to 'ID_spatial_unit'. 

In [ ]:
# I'm reading in the raw ZCTA data. 
zctas = gpd.read_parquet(data_dir / "01_raw_data" / "zctas_2020.parquet")
# Rename 
zctas.rename(columns={"zcta": "ID_spatial_unit"}, inplace=True)
zctas.head

Then, I'll save this as a geojson file as well. This additional spatial unit 
file can be in geojson or parquet format. 

In [ ]:
# This will take a minute. 
zctas_path = data_dir / "02_interim_data" / "zctas_2020.geojson"
zctas.to_file(zctas_path, driver = 'GeoJSON')


Since the gridded population raster doesn't require any preprocessing, our 
data is ready! Proceed to 02_run_example.ipynb to continue the tutorial. 